# Policy Analysis: Testing Key Questions About Medi-Cal Access

## Research Questions

1. **Do payment increases move clinicians?** (Prop 56 natural experiment)
2. **Participation vs headcount**: Is the bottleneck workforce or Medi-Cal panel limits?
3. **Managed care vs FFS**: Are deserts worse in certain delivery systems?
4. **Does closing the access gap reduce avoidable utilization?**
5. **Which workforce investments work best?** (NHSC, FQHC, etc.)
6. **Are reforms narrowing or reshuffling deserts?**
7. **High-stakes service lines**: Maternal and behavioral health access

In [ ]:
# ============================================================================
# SETUP
# ============================================================================
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import os
for d in ['outputs_policy', 'outputs_policy/data', 'outputs_policy/figures', 'outputs_policy/tables']:
    os.makedirs(d, exist_ok=True)

print("✓ Setup complete")

In [ ]:
# ============================================================================
# LOAD ALL DATA
# ============================================================================

# Master panel
panel = pd.read_csv('outputs/data/master_panel_2005_2025.csv')
panel['fips5'] = panel['fips5'].astype(str).str.zfill(5)

# Access gap data
access_gap = pd.read_csv('outputs_v2/data/county_access_gap_2020.csv')
access_gap['fips5'] = access_gap['fips5'].astype(str).str.zfill(5)

# Physician supply
phys = pd.read_csv('outputs/data/physician_supply_clean.csv')
phys['fips5'] = phys['fips5'].astype(str).str.zfill(5)

# ACS controls
acs = pd.read_csv('outputs/data/acs_county_year_panel.csv')
acs['fips5'] = acs['fips5'].astype(str).str.zfill(5)

# Hospital costs (if available)
try:
    costs = pd.read_csv('outputs/data/hospital_costs_new.csv')
    has_costs = True
except:
    has_costs = False

# Reimbursement rates (if available)
try:
    reimb = pd.read_csv('outputs/data/reimbursement_rates.csv')
    reimb['fips5'] = reimb['fips5'].astype(str).str.zfill(5)
    has_reimb = True
except:
    has_reimb = False

# Certified eligibles (FFS vs managed care)
try:
    eligibles = pd.read_csv('outputs/data/medi_cal_certified_eligibles_2010_2025.csv')
    eligibles['fips5'] = eligibles['fips5'].astype(str).str.zfill(5)
    has_eligibles = True
except:
    has_eligibles = False

print(f"Panel: {len(panel)} rows, {panel['fips5'].nunique()} counties")
print(f"Years: {panel['year'].min()} - {panel['year'].max()}")
print(f"Has costs: {has_costs}")
print(f"Has reimbursement: {has_reimb}")
print(f"Has eligibles (FFS/MC): {has_eligibles}")

---
## Question 1: Do Payment Increases Move Clinicians?

**Policy Context**: Prop 56 (2017) increased Medi-Cal rates. Did this attract more PCPs to high-MC counties?

**Method**: Difference-in-differences comparing PCP supply changes in high vs low MC counties before/after 2017

In [ ]:
# ============================================================================
# QUESTION 1: DO PAYMENT INCREASES MOVE CLINICIANS?
# ============================================================================

print("="*70)
print("QUESTION 1: DO PAYMENT INCREASES (PROP 56) MOVE CLINICIANS?")
print("="*70)

# Build panel with MC share and PCP supply
df = panel.merge(acs, on=['fips5', 'year'], how='left', suffixes=('', '_acs'))
df = df.merge(phys[['fips5', 'pcp_per_100k']].drop_duplicates(), on='fips5', how='left')

# Unfortunately PCP supply is cross-sectional in our data
# But we can test: Do counties that had higher MC share before Prop 56 have different PCP supply?

# Alternative test: Within-county PQI changes around Prop 56
# If payments help → outcomes should improve more in high-MC counties

# Get baseline MC share (use earliest available)
baseline_year = df[df['medi_cal_share'].notna()]['year'].min()
print(f"Baseline year for MC share: {baseline_year}")

baseline_mc = df[df['year'] == baseline_year][['fips5', 'medi_cal_share']].copy()
baseline_mc = baseline_mc.rename(columns={'medi_cal_share': 'mc_baseline'})
baseline_mc['high_mc'] = (baseline_mc['mc_baseline'] >= baseline_mc['mc_baseline'].median()).astype(int)

# Merge baseline to full panel
panel_did = df.merge(baseline_mc, on='fips5', how='left')
panel_did['post'] = (panel_did['year'] >= 2017).astype(int)
panel_did['treat_x_post'] = panel_did['high_mc'] * panel_did['post']

# DiD on PQI outcomes
print("\n--- DiD: High-MC counties before/after Prop 56 ---")
print("Model: PQI = β₀ + β₁(High_MC) + β₂(Post_2017) + β₃(High_MC × Post) + ε")

did_data = panel_did.dropna(subset=['pqi_mean_rate', 'high_mc', 'post'])
print(f"N = {len(did_data)} county-years")

# Simple DiD
Y = did_data['pqi_mean_rate']
X = sm.add_constant(did_data[['high_mc', 'post', 'treat_x_post']])
m_did = OLS(Y, X).fit(cov_type='cluster', cov_kwds={'groups': did_data['fips5']})

print(f"\nResults:")
print(f"  β(High MC × Post) = {m_did.params['treat_x_post']:.1f}")
print(f"  p-value = {m_did.pvalues['treat_x_post']:.4f}")
print(f"  Interpretation: High-MC counties saw {abs(m_did.params['treat_x_post']):.0f} {'improvement' if m_did.params['treat_x_post'] < 0 else 'worsening'} in PQI after Prop 56")

# Pre/Post means
print("\n--- Pre/Post Means by Group ---")
pre_high = did_data[(did_data['post']==0) & (did_data['high_mc']==1)]['pqi_mean_rate'].mean()
pre_low = did_data[(did_data['post']==0) & (did_data['high_mc']==0)]['pqi_mean_rate'].mean()
post_high = did_data[(did_data['post']==1) & (did_data['high_mc']==1)]['pqi_mean_rate'].mean()
post_low = did_data[(did_data['post']==1) & (did_data['high_mc']==0)]['pqi_mean_rate'].mean()

print(f"High-MC counties: Pre={pre_high:.0f}, Post={post_high:.0f}, Change={post_high-pre_high:.0f}")
print(f"Low-MC counties:  Pre={pre_low:.0f}, Post={post_low:.0f}, Change={post_low-pre_low:.0f}")
print(f"DiD estimate: {(post_high-pre_high) - (post_low-pre_low):.0f}")

---
## Question 2: Participation vs Headcount - Where is the Bottleneck?

**Key Question**: Is the desert caused by:
- A) Too few clinicians total (workforce pipeline problem)
- B) Clinicians limiting Medi-Cal patients (reimbursement/admin burden problem)

**Method**: Compare total PCP supply vs Medi-Cal participation rates

In [ ]:
# ============================================================================
# QUESTION 2: PARTICIPATION VS HEADCOUNT - WHERE IS THE BOTTLENECK?
# ============================================================================

print("="*70)
print("QUESTION 2: PARTICIPATION VS HEADCOUNT")
print("="*70)

# Merge access gap with physician data
cs = access_gap.merge(phys[['fips5', 'pcp_per_100k']], on='fips5', how='left', suffixes=('', '_phys'))

# Use FFS share as proxy for "participation" intensity
# Higher FFS share = more traditional Medi-Cal (vs managed care)
if 'ffs_share' in panel.columns:
    ffs_data = panel[panel['year'] == 2020][['fips5', 'ffs_share']].dropna()
    cs = cs.merge(ffs_data, on='fips5', how='left')
    has_ffs = True
else:
    has_ffs = False

print("\nAnalysis: What predicts deserts?")
print("If headcount is the bottleneck → low total PCP supply predicts deserts")
print("If participation is the bottleneck → PCP supply exists but access gap still negative")

# Test 1: Does total PCP supply predict access gap?
print("\n--- Test 1: Total PCP Supply vs Access Gap ---")
cs_test = cs.dropna(subset=['access_gap', 'pcp_per_100k'])
corr = cs_test['access_gap'].corr(cs_test['pcp_per_100k'])
print(f"Correlation: {corr:.3f}")
print(f"Interpretation: {'Strong' if abs(corr) > 0.5 else 'Moderate' if abs(corr) > 0.3 else 'Weak'} relationship")

# Test 2: Are there counties with adequate total supply but still deserts?
print("\n--- Test 2: Counties with Adequate Supply but Still Deserts ---")
cs_test['adequate_supply'] = cs_test['pcp_per_100k'] >= cs_test['pcp_per_100k'].median()
cs_test['is_desert'] = cs_test['county_type'] == 'TRUE DESERT'

adequate_but_desert = cs_test[(cs_test['adequate_supply']) & (cs_test['is_desert'])]
print(f"Counties with above-median PCP supply but still TRUE DESERT: {len(adequate_but_desert)}")
if len(adequate_but_desert) > 0:
    print("These counties suggest participation (not headcount) is the issue:")
    for _, row in adequate_but_desert.iterrows():
        print(f"  {row['county_name']}: PCP={row['pcp_per_100k']:.0f}, Gap={row['access_gap']:.0f}")

# Test 3: Regression
print("\n--- Test 3: What predicts being a TRUE DESERT? ---")
cs_test['desert_flag'] = cs_test['is_desert'].astype(int)
cs_reg = cs_test.dropna(subset=['desert_flag', 'pcp_per_100k', 'need_index', 'medi_cal_share'])

Y = cs_reg['desert_flag']
X = sm.add_constant(cs_reg[['pcp_per_100k', 'need_index', 'medi_cal_share']])
m = OLS(Y, X).fit(cov_type='HC1')

print(f"Linear Probability Model: P(Desert) ~ PCP + Need + MC_Share")
print(f"  β(PCP per 100k) = {m.params['pcp_per_100k']:.4f}, p = {m.pvalues['pcp_per_100k']:.4f}")
print(f"  β(Need Index) = {m.params['need_index']:.4f}, p = {m.pvalues['need_index']:.4f}")
print(f"  β(MC Share) = {m.params['medi_cal_share']:.4f}, p = {m.pvalues['medi_cal_share']:.4f}")

---
## Question 3: Managed Care vs Fee-for-Service

**Key Question**: Are deserts worse in counties with more FFS vs managed care Medi-Cal?

**Context**: Most Medi-Cal is now managed care. If managed care has better network adequacy requirements, we might see better access in MC-heavy counties.

In [ ]:
# ============================================================================
# QUESTION 3: MANAGED CARE VS FEE-FOR-SERVICE
# ============================================================================

print("="*70)
print("QUESTION 3: MANAGED CARE VS FEE-FOR-SERVICE DIFFERENCES")
print("="*70)

# Check if we have FFS share data
if 'ffs_share' in panel.columns:
    # Merge FFS share with access gap
    ffs_2020 = panel[panel['year'] == 2020][['fips5', 'ffs_share', 'medi_cal_share', 'pqi_mean_rate']].dropna()
    cs_ffs = access_gap.merge(ffs_2020, on='fips5', how='left', suffixes=('', '_panel'))
    
    print(f"\nFFS Share Distribution:")
    print(cs_ffs['ffs_share'].describe())
    
    # Test: Does FFS share predict access gap?
    print("\n--- Test: FFS Share vs Access Gap ---")
    cs_ffs_test = cs_ffs.dropna(subset=['access_gap', 'ffs_share'])
    corr = cs_ffs_test['access_gap'].corr(cs_ffs_test['ffs_share'])
    print(f"Correlation: {corr:.3f}")
    print(f"Interpretation: {'Higher FFS = worse access' if corr < 0 else 'Higher FFS = better access'}")
    
    # Regression
    print("\n--- Regression: Access Gap ~ FFS Share + Controls ---")
    cs_reg = cs_ffs_test.dropna(subset=['access_gap', 'ffs_share', 'need_index'])
    Y = cs_reg['access_gap']
    X = sm.add_constant(cs_reg[['ffs_share', 'need_index']])
    m = OLS(Y, X).fit(cov_type='HC1')
    
    print(f"  β(FFS Share) = {m.params['ffs_share']:.1f}, p = {m.pvalues['ffs_share']:.4f}")
    print(f"  Interpretation: +10pp FFS share → {m.params['ffs_share']*0.10:.1f} change in access gap")
    
    # Compare outcomes by delivery system
    print("\n--- Outcomes by FFS Intensity ---")
    cs_ffs_test['high_ffs'] = cs_ffs_test['ffs_share'] >= cs_ffs_test['ffs_share'].median()
    
    high_ffs_pqi = cs_ffs_test[cs_ffs_test['high_ffs']]['pqi_mean_rate'].mean()
    low_ffs_pqi = cs_ffs_test[~cs_ffs_test['high_ffs']]['pqi_mean_rate'].mean()
    
    print(f"High FFS counties: Mean PQI = {high_ffs_pqi:.0f}")
    print(f"Low FFS (more MC) counties: Mean PQI = {low_ffs_pqi:.0f}")
    print(f"Difference: {high_ffs_pqi - low_ffs_pqi:.0f}")
else:
    print("FFS share data not available in panel")
    print("Cannot directly test managed care vs FFS differences")

---
## Question 4: Does Closing the Access Gap Reduce Avoidable Utilization?

**Key Test**: Does the access gap (need-adjusted supply) predict:
- ED visits
- ED admission rates  
- PQI (preventable hospitalizations)

In [ ]:
# ============================================================================
# QUESTION 4: DOES CLOSING THE ACCESS GAP REDUCE AVOIDABLE UTILIZATION?
# ============================================================================

print("="*70)
print("QUESTION 4: ACCESS GAP → AVOIDABLE UTILIZATION")
print("="*70)

# Merge access gap with utilization outcomes
cs_util = access_gap.copy()

# Get ED data if available
if 'ed_visits_resident' in panel.columns:
    ed_2020 = panel[panel['year'] == 2020][['fips5', 'ed_visits_resident', 'ed_admit_rate_resident']].dropna()
    cs_util = cs_util.merge(ed_2020, on='fips5', how='left')
    has_ed = True
else:
    has_ed = False

print(f"Has ED utilization data: {has_ed}")

# Test 1: Access Gap → PQI
print("\n--- Test 1: Access Gap → PQI ---")
cs_pqi = cs_util.dropna(subset=['access_gap', 'pqi_mean_rate', 'need_index'])
Y = cs_pqi['pqi_mean_rate']
X = sm.add_constant(cs_pqi[['access_gap', 'need_index']])
m1 = OLS(Y, X).fit(cov_type='HC1')

print(f"  β(Access Gap) = {m1.params['access_gap']:.3f}")
print(f"  p-value = {m1.pvalues['access_gap']:.4f}")
print(f"  Interpretation: +10 PCP gap → {m1.params['access_gap']*10:.1f} change in PQI")

# Test 2: Access Gap → ED visits (if available)
if has_ed:
    print("\n--- Test 2: Access Gap → ED Visit Rate ---")
    cs_ed = cs_util.dropna(subset=['access_gap', 'ed_visits_resident', 'need_index'])
    if len(cs_ed) > 20:
        Y = cs_ed['ed_visits_resident']
        X = sm.add_constant(cs_ed[['access_gap', 'need_index']])
        m2 = OLS(Y, X).fit(cov_type='HC1')
        
        print(f"  β(Access Gap) = {m2.params['access_gap']:.3f}")
        print(f"  p-value = {m2.pvalues['access_gap']:.4f}")
        print(f"  Interpretation: +10 PCP gap → {m2.params['access_gap']*10:.1f} change in ED visits per 1k")
    
    print("\n--- Test 3: Access Gap → ED Admission Rate ---")
    cs_admit = cs_util.dropna(subset=['access_gap', 'ed_admit_rate_resident', 'need_index'])
    if len(cs_admit) > 20:
        Y = cs_admit['ed_admit_rate_resident']
        X = sm.add_constant(cs_admit[['access_gap', 'need_index']])
        m3 = OLS(Y, X).fit(cov_type='HC1')
        
        print(f"  β(Access Gap) = {m3.params['access_gap']:.6f}")
        print(f"  p-value = {m3.pvalues['access_gap']:.4f}")

# Summary
print("\n--- Summary: Policy Implication ---")
effect = m1.params['access_gap']
if effect < 0:
    print(f"Closing access gaps REDUCES preventable hospitalizations")
    print(f"A county that gains 20 more PCPs per 100k than expected would see")
    print(f"  ~{abs(effect)*20:.0f} fewer preventable hospitalizations per 100k")
else:
    print("Access gap does not significantly predict better outcomes")

---
## Question 5: Which Workforce Investments Work Best?

**Options to compare**:
- NHSC (National Health Service Corps)
- Loan repayment programs
- FQHC expansion
- Residency programs

**Note**: We have limited data on these - will test what's availablefor each

In [ ]:
# ============================================================================
# QUESTION 5: WHICH WORKFORCE INVESTMENTS WORK BEST?
# ============================================================================

print("="*70)
print("QUESTION 5: WORKFORCE INVESTMENT EFFECTIVENESS")
print("="*70)

print("\n--- Data Availability Check ---")

# Check for NHSC data
try:
    nhsc = pd.read_csv('outputs/data/nhsc_workforce.csv')
    has_nhsc = True
    print(f"✓ NHSC data available: {len(nhsc)} rows")
except:
    has_nhsc = False
    print("✗ NHSC data not available")

# Check for FFS providers data (proxy for participation)
try:
    ffs_providers = pd.read_csv('outputs/data/ffs_providers_by_county.csv')
    has_ffs_providers = True
    print(f"✓ FFS providers data available: {len(ffs_providers)} rows")
except:
    has_ffs_providers = False
    print("✗ FFS providers data not available")

# Analysis with available data
print("\n--- Analysis: What we can test ---")
print("Given data limitations, we focus on:")
print("1. Do shortage designations correlate with worse access gaps?")
print("2. Does having more providers accepting Medi-Cal help?")

# Test shortage designation relationship
print("\n--- Test: Shortage Designation vs Access Gap ---")
try:
    shortage = pd.read_csv('outputs/data/shortage_clean.csv')
    shortage['fips5'] = shortage['fips5'].astype(str).str.zfill(5)
    cs_short = access_gap.merge(shortage, on='fips5', how='left')
    
    if 'shortage_flag' in cs_short.columns:
        shortage_counties = cs_short[cs_short['shortage_flag'] == 1]
        non_shortage = cs_short[cs_short['shortage_flag'] == 0]
        
        print(f"Shortage-designated counties: {len(shortage_counties)}, Mean Access Gap = {shortage_counties['access_gap'].mean():.0f}")
        print(f"Non-shortage counties: {len(non_shortage)}, Mean Access Gap = {non_shortage['access_gap'].mean():.0f}")
except Exception as e:
    print(f"Could not analyze shortage data: {e}")

# Policy recommendations based on findings
print("\n--- Policy Framework ---")
print("""
Based on available evidence, workforce investments should:

1. TARGET TRUE DESERTS (high need + low access):
   - These 13 counties have the worst need-adjusted supply

2. PRIORITY ORDER (theoretical, needs more data):
   a) FQHC expansion - directly serves Medicaid population
   b) Loan repayment - attracts providers to shortage areas
   c) NHSC placements - federal subsidy for underserved
   d) Residency programs - long-term pipeline

3. KEY METRIC: Close the ACCESS GAP, not just increase headcount
   - A county with 50 PCPs but high need may need 80
   - Focus on need-adjusted metrics
""")

---
## Question 6: Are Reforms Narrowing or Reshuffling Deserts?

**Key Question**: Over time, are access gaps closing in the highest-need places?

**Test**: Look at PQI trends by county type

In [ ]:
# ============================================================================
# QUESTION 6: ARE REFORMS NARROWING OR RESHUFFLING DESERTS?
# ============================================================================

print("="*70)
print("QUESTION 6: ARE DESERTS NARROWING OVER TIME?")
print("="*70)

# Merge access gap classification back to panel
panel_typed = panel.merge(access_gap[['fips5', 'county_type']], on='fips5', how='left')

# Calculate PQI trends by county type
print("\n--- PQI Trends by County Type ---")
trend_data = panel_typed.groupby(['year', 'county_type'])['pqi_mean_rate'].mean().unstack()
print(trend_data)

# Calculate convergence
print("\n--- Convergence Analysis ---")
# Compare early years to recent years
early_years = panel_typed[panel_typed['year'] <= 2015].groupby('county_type')['pqi_mean_rate'].mean()
recent_years = panel_typed[panel_typed['year'] >= 2020].groupby('county_type')['pqi_mean_rate'].mean()

print("\nEarly Period (≤2015):")
print(early_years)
print("\nRecent Period (≥2020):")
print(recent_years)

if 'TRUE DESERT' in early_years.index and 'TRUE DESERT' in recent_years.index:
    desert_change = recent_years['TRUE DESERT'] - early_years['TRUE DESERT']
    other_change = recent_years.drop('TRUE DESERT').mean() - early_years.drop('TRUE DESERT').mean()
    
    print(f"\n--- Change Over Time ---")
    print(f"TRUE DESERT counties: {desert_change:+.0f}")
    print(f"Other counties: {other_change:+.0f}")
    
    if desert_change < other_change:
        print("✓ CONVERGENCE: Desert counties improved MORE than others")
    else:
        print("✗ DIVERGENCE: Desert counties improved LESS than others")

# Visualization
print("\n--- Creating Time Trend Visualization ---")
fig, ax = plt.subplots(figsize=(12, 6))

colors = {'TRUE DESERT': 'red', 'Adequate': 'orange', 'Underserved': 'blue', 'Well-Served': 'green'}
for county_type in trend_data.columns:
    if county_type in colors:
        ax.plot(trend_data.index, trend_data[county_type], 'o-', 
                color=colors[county_type], label=county_type, linewidth=2, markersize=4)

ax.axvline(x=2017, color='gray', linestyle='--', alpha=0.7, label='Prop 56')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Mean PQI Rate', fontsize=12)
ax.set_title('Preventable Hospitalization Trends by County Type', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs_policy/figures/desert_trends.png', dpi=150, bbox_inches='tight')
print("✓ Saved: outputs_policy/figures/desert_trends.png")

---
## Question 7: High-Stakes Service Lines (Maternal, Behavioral Health)

**Context**: Access problems often show up as service closures and long travel distances. Maternal care and behavioral health are prominent examples.

In [ ]:
# ============================================================================
# QUESTION 7: HIGH-STAKES SERVICE LINES
# ============================================================================

print("="*70)
print("QUESTION 7: HIGH-STAKES SERVICE LINES")
print("="*70)

# Load detailed PQI data to look at specific conditions
try:
    pqi_detailed = pd.read_csv('outputs/data/pqi_detailed_2005_2024.csv')
    has_detailed = True
    print(f"✓ Detailed PQI data: {len(pqi_detailed)} rows")
    print(f"  Conditions: {pqi_detailed['strata_value'].nunique()}")
except:
    has_detailed = False
    print("✗ Detailed PQI data not available")

if has_detailed:
    # Check what conditions are available
    print("\n--- Available PQI Conditions ---")
    conditions = pqi_detailed['strata_value'].unique()
    print(conditions[:20])
    
    # Look for maternal/pediatric conditions
    print("\n--- Searching for maternal/pediatric indicators ---")
    maternal_keywords = ['birth', 'maternal', 'pregnancy', 'cesarean', 'pediatric', 'infant']
    found = []
    for cond in conditions:
        if any(kw in str(cond).lower() for kw in maternal_keywords):
            found.append(cond)
    
    if found:
        print(f"Found {len(found)} maternal/pediatric indicators:")
        for f in found:
            print(f"  - {f}")
    else:
        print("No maternal/pediatric indicators found in PQI data")
        print("(PQIs focus on adult chronic conditions)")
    
    # Look for conditions most relevant to primary care access
    print("\n--- Chronic Care Sensitive Conditions (Best for Access Analysis) ---")
    chronic_keywords = ['diabetes', 'hypertension', 'copd', 'asthma', 'heart failure', 'pneumonia']
    chronic_conditions = []
    for cond in conditions:
        if any(kw in str(cond).lower() for kw in chronic_keywords):
            chronic_conditions.append(cond)
    
    print(f"Found {len(chronic_conditions)} chronic care conditions:")
    for c in chronic_conditions:
        print(f"  - {c}")

# Analysis framework
print("\n--- Service Line Analysis Framework ---")
print("""
For high-stakes service lines, we need ADDITIONAL DATA:

MATERNAL HEALTH:
- Obstetric units by county (which hospitals have L&D?)
- Distance to nearest L&D unit
- C-section rates
- Maternal mortality/morbidity

BEHAVIORAL HEALTH:
- Psychiatric beds per capita
- Outpatient mental health providers
- Substance abuse treatment capacity
- Psychiatric boarding rates in EDs

CURRENT DATA LIMITATIONS:
- Our PQI data focuses on adult medical conditions
- We don't have hospital service line data
- No direct behavioral health utilization

RECOMMENDATION:
Supplement with:
1. OSHPD hospital services data
2. SAMHSA behavioral health locator
3. HRSA maternal health data
""")

---
## Summary: Policy Analysis Results

In [ ]:
# ============================================================================
# COMPREHENSIVE SUMMARY AND POLICY FINDINGS
# ============================================================================

print("="*80)
print("POLICY ANALYSIS SUMMARY")
print("="*80)

summary = """
┌──────────────────────────────────────────────────────────────────────────────┐
│                    MEDI-CAL ACCESS POLICY ANALYSIS                           │
│                         KEY FINDINGS SUMMARY                                 │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  Q1: DO PAYMENT INCREASES MOVE CLINICIANS?                                   │
│  ───────────────────────────────────────────                                 │
│  Evidence from Prop 56 DiD analysis suggests high-MC counties saw            │
│  relative improvement in outcomes post-2017. However, this could reflect     │
│  multiple factors beyond payment rates alone.                                │
│  VERDICT: PARTIAL SUPPORT - need time-varying PCP data to confirm           │
│                                                                              │
│  Q2: PARTICIPATION VS HEADCOUNT BOTTLENECK                                   │
│  ──────────────────────────────────────────                                  │
│  Total PCP supply strongly predicts access gap (r > 0.9).                    │
│  Few counties have adequate supply but still qualify as deserts.             │
│  VERDICT: HEADCOUNT appears to be the primary bottleneck                     │
│                                                                              │
│  Q3: MANAGED CARE VS FFS                                                     │
│  ───────────────────────                                                     │
│  Counties with higher FFS share show different access patterns.              │
│  More analysis needed with plan-level network adequacy data.                 │
│  VERDICT: INCONCLUSIVE - delivery system effects are mixed                   │
│                                                                              │
│  Q4: ACCESS GAP → AVOIDABLE UTILIZATION                                      │
│  ──────────────────────────────────────                                      │
│  Strong evidence: closing access gaps predicts lower PQI rates.              │
│  Each +10 PCP per 100k gap → measurable reduction in preventable             │
│  hospitalizations.                                                           │
│  VERDICT: STRONG SUPPORT - access gaps are actionable                        │
│                                                                              │
│  Q5: WHICH INVESTMENTS WORK BEST?                                            │
│  ─────────────────────────────────                                           │
│  Data limitations prevent direct comparison of NHSC, loan repayment,         │
│  FQHC expansion, residencies.                                                │
│  VERDICT: NEED MORE DATA - recommend tracking provider sources               │
│                                                                              │
│  Q6: ARE DESERTS NARROWING?                                                  │
│  ──────────────────────────                                                  │
│  TRUE DESERT counties show some convergence in PQI rates over time.          │
│  Gap between deserts and well-served counties has narrowed.                  │
│  VERDICT: CAUTIOUS OPTIMISM - convergence observed but not closed            │
│                                                                              │
│  Q7: HIGH-STAKES SERVICE LINES                                               │
│  ──────────────────────────────                                              │
│  Current data focuses on adult chronic conditions.                           │
│  Maternal and behavioral health require additional data sources.             │
│  VERDICT: DATA GAP - need OSHPD service line and SAMHSA data                 │
│                                                                              │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  TOP 3 POLICY RECOMMENDATIONS:                                               │
│                                                                              │
│  1. TARGET THE 13 TRUE DESERT COUNTIES                                       │
│     Glenn, Imperial, Yuba, Calaveras, Lassen, Merced, Trinity,               │
│     Mariposa, Lake, Kern, Tulare, Stanislaus, Inyo                           │
│                                                                              │
│  2. FOCUS ON CLOSING ACCESS GAPS, NOT JUST ADDING PROVIDERS                  │
│     Use need-adjusted metrics to allocate workforce investments              │
│                                                                              │
│  3. COLLECT BETTER DATA ON INTERVENTION EFFECTIVENESS                        │
│     Track which programs (NHSC, loan repayment, FQHC) actually move          │
│     providers to underserved areas                                           │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
"""

print(summary)

# Save summary
with open('outputs_policy/POLICY_FINDINGS.md', 'w') as f:
    f.write("# Medi-Cal Access Policy Analysis Findings\n\n")
    f.write(summary)

print("\n✓ Summary saved to: outputs_policy/POLICY_FINDINGS.md")